# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [ ]:
# Install and Import Required Modules
%pip install langchain_experimental faiss-cpu openai python-dotenv

from typing import Optional
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import os
import numpy as np
import faiss
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the .env file
load_dotenv(find_dotenv())

# Retrieve API keys from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

# Initialize the OpenAI embedding model
embeddings_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

# Define an embedding size (OpenAI uses 1536 dimensions for its embeddings)
embedding_size = 1536

# Initialize FAISS index with L2 distance metric
index = faiss.IndexFlatL2(embedding_size)

# Example: Embedding and adding a query
query = "What are the latest trends in AI?"
embedding_vector = embeddings_model.embed_query(query)

# Convert the vector to a 2D numpy array (required by FAISS)
vector = np.array([embedding_vector])

# Add the vector to the FAISS index
index.add(vector)

# Initialize the InMemoryDocstore with the document metadata
docstore = InMemoryDocstore({"doc_id_1": {"content": query}})

# Initialize FAISS vector store
vectorstore = FAISS(embeddings_model, index, docstore, {})

print("Vector store is set up and ready for use.")


Note: you may need to restart the kernel to use updated packages.
Vector store is set up and ready for use.


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [21]:
OBJECTIVE = "Write a weather report for SF today"

In [41]:
# Required imports
from langchain_experimental.autonomous_agents import BabyAGI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
import faiss
import os

# Ensure environment variables are loaded
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI LLM
llm = OpenAI(api_key=OPENAI_API_KEY)

# Define necessary components

# Task Creation Chain
task_creation_prompt = PromptTemplate.from_template("Create tasks based on the objective: {objective}")
task_creation_chain = LLMChain(prompt=task_creation_prompt, llm=llm, output_key="task_creation")

# Task Prioritization Chain
task_prioritization_prompt = PromptTemplate.from_template("Prioritize tasks based on their importance: {objective}")
task_prioritization_chain = LLMChain(prompt=task_prioritization_prompt, llm=llm, output_key="task_prioritization")

# Execution Chain
execution_prompt = PromptTemplate.from_template("Execute tasks to achieve the objective: {objective}")
execution_chain = LLMChain(prompt=execution_prompt, llm=llm, output_key="execution")

# Set up the Vector Store (FAISS)
embedding_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)

# Initialize InMemoryDocstore for storing document metadata
docstore = InMemoryDocstore({})

# Initialize FAISS vector store
vectorstore = FAISS(embedding_model, index, docstore, {})

# Define the objective
OBJECTIVE = "Write a weather report for SF today"

# Create the BabyAGI instance
baby_agi = BabyAGI(
    objective=OBJECTIVE,
    task_creation_chain=task_creation_chain,
    task_prioritization_chain=task_prioritization_chain,
    execution_chain=execution_chain,
    vectorstore=vectorstore
)

# Function to check task ID and handle execution
def execute_task_with_id_check(task):
    try:
        # Validate if 'task_id' exists and is a valid integer
        if 'task_id' in task and task['task_id'].isdigit():
            this_task_id = int(task['task_id'])
        else:
            raise ValueError(f"Invalid task_id: {task.get('task_id', 'Missing task_id')}")

        # Execute the task and return the result
        result = baby_agi.execute_task(OBJECTIVE, task["task_name"])
        return result

    except ValueError as ve:
        print(f"Error: {ve}")
        return None  # Return None to gracefully handle errors

# Run the BabyAGI agent with task validation
def run_baby_agi():
    try:
        # Get the initial task
        tasks = [{"task_id": "1", "task_name": "Gather weather data"}]  # Example task list
        for task in tasks:
            result = execute_task_with_id_check(task)
            if result:
                print(result)
            else:
                print("Task execution skipped due to error.")
    except Exception as e:
        print(f"An error occurred during BabyAGI execution: {e}")

# Run the agent
run_baby_agi()




1. Start by gathering information about the current weather in San Francisco. This can be done by checking local weather websites, news channels, or using a weather app.

2. Begin the report by stating the date and time. For example, "Good morning San Francisco, it is Wednesday, September 1st, and the time is 9:00 AM."

3. Mention the current temperature in the city. You can say something like, "The temperature in San Francisco is currently 65 degrees Fahrenheit."

4. Describe the overall weather conditions. Is it sunny, cloudy, or partly cloudy? Is there any wind or rain? Use descriptive language to paint a picture for the audience. For example, "It is a beautiful day in San Francisco with clear blue skies and a light breeze."

5. Provide information about the expected high and low temperatures for the day. This will give listeners an idea of how hot or cold it will get throughout the day. For instance, "We can expect a high of 75 degrees and a low of 60 degrees later tonight."

6. 